In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection"
CSM = CarlaScenesManager(data_dir=data_root, split="train")
CDM = CSM.get_scene_dataset_by_index(0)

In [ ]:
from avstack.modules.perception.fov_estimator import FastRayTraceBevLidarFovEstimator
from avstack.modules.perception.fov_estimator import SlowRayTraceBevLidarFovEstimator

# set up fov models
fov_estimator_fast = FastRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)
fov_estimator_slow = SlowRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)

# get some random lidar point clouds to analyze
seed = 10
rng = np.random.RandomState(seed=seed)
n_pcs = 20
pcs = []
frames = CDM.get_frames(sensor="lidar-0", agent=0)
random_agent_frames = list(zip(rng.randint(0, 4, size=n_pcs), rng.choice(list(range(200)), size=n_pcs, replace=False)))
pcs = [CDM.get_lidar(frame=frames[idx_frame], sensor="lidar-0", agent=agent) for agent, idx_frame in random_agent_frames]

In [ ]:
for pc in pcs:
    # run fov estimator on this frame
    fov_hull = pc.concave_hull_bev(concavity=1)
    fov_fast = fov_estimator_fast(pc)
    fov_slow = fov_estimator_slow(pc)
    
    # project lidar to bev for plotting
    pc_bev = pc.project_to_2d_bev()
    
    # make plot
    plt.plot(fov_hull.boundary[:,0], fov_hull.boundary[:,1], "r-", label="Hull FOV Model")
    plt.plot(fov_fast.boundary[:,0], fov_fast.boundary[:,1], "k--", label="Fast FOV Model")
    plt.plot(fov_slow.boundary[:,0], fov_slow.boundary[:,1], "g-", label="Slow FOV Model")
    plt.plot(pc_bev.data.x[:,0], pc_bev.data.x[:,1], "bo", markersize=2, label="Point Cloud")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.show()

In [ ]:
# run execution timing check
n_pcs_timing = 10
%timeit for pc in pcs[:n_pcs_timing]:fov_estimator_fast(pc)
%timeit for pc in pcs[:n_pcs_timing]:fov_estimator_slow(pc)